In [11]:
import numpy as np
import skimage.io as io
from skimage.color import rgb2gray
from skimage.transform import hough_line, hough_line_peaks, rotate
from skimage.feature import canny
from skimage import data
from skimage.filters import threshold_otsu

In [16]:
def get_max_freq(list):
    freq = {} 
    for item in list: 
        if (item in freq): 
            freq[item] += 1
        else: 
            freq[item] = 1
    wanted_value=0
    wanted_angle=0
    for key, value in freq.items():
        if wanted_value<value:
            wanted_value=value
            wanted_angle=key       
    return wanted_angle       

In [20]:
def Derotate(bw_image):
    # hough line to detect lines in the photo
    tested_angles = np.linspace(-np.pi / 2, np.pi / 2, 360)
    h, theta, d = hough_line(bw_image, theta=tested_angles)
    io.imshow(bw_image)
    origin = np.array((0, bw_image.shape[1]))
    # hough peaks to get those lines
    angles=[]
    for _, angle, dist in zip(*hough_line_peaks(h, theta, d)):
        y0, y1 = (dist - origin * np.cos(angle)) / np.sin(angle)
        angles.append(angle)
    angles = [angle * 180 / np.pi for angle in angles]
    # git the most repeated angle on photo
    rotating_angle=get_max_freq(angles)
    # rotate photo 
    if rotating_angle>0:
        rotated_image=rotate(bw_image,rotating_angle-90,resize=False)
    else:
        rotated_image=rotate(bw_image,rotating_angle+90,resize=False)
    return rotated_image

In [23]:
def Deskew(img):
    # Dilation to make a rectangle around staff lines for 
    Structure_Element=np.ones((8,8))
    dilated_img=binary_dilation(img, Structure_Element, out=None)
    # find contour for the found rectangle
    find_contour_img=find_contours(dilated_img,0.8)
    fig, ay = plt.subplots()
    final= np.zeros(img.shape)
    X1=0
    x2=0
    y1=0
    y2=0
    # getthe diminsion to use prespective projection
    for box in find_contour_img:  
        Xmin=min(box[:,1])
        Xmax=max(box[:,1])
        Ymin=min(box[:,0])
        Ymax=max(box[:,0])
        ratio=(Ymax-Ymin)/(Xmax-Xmin)
        if ratio >=0.3 and ratio<0.4:
            x1=Xmin
            y1=Ymin
            x2=Xmax
            y2=Ymax
    h=img.shape[0]
    w=img.shape[1]
    # prespective projection
    pts1 = np.float32([[x1,y1],[x2,y1],[x1,y2],[x2,y2]])
    pts2 = np.float32([[0,0],[w,0],[0,h],[w,h]])
    M = cv2.getPerspectiveTransform(pts1,pts2)
    dst = cv2.warpPerspective(img,M,(w,h))
    return dst
    

In [26]:
image=io.imread('rotated.jpg')
gray_image=rgb2gray(image)
threshold=threshold_otsu(gray_image)
bw_image=gray_image <=threshold
io.imshow(bw_image)
io.show()
rotated_image=Derotate(bw_image)
io.imshow(rotated_image)
io.show()
skewed_img=Deskew(rotated_image)
io.imshow(skewed_img)
io.show()

FileNotFoundError: No such file: 'd:\CMP\CMP 301\IP\project\work\OMR-Image-Processing\preprocessing\rotated.jpg'